# Análise de sentimentos

## Importação das bibliotecas e base de dados

- Base de dados: https://www.kaggle.com/sid321axn/amazon-alexa-reviews/kernels

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
reviews_df = pd.read_csv('/content/amazon_alexa.tsv', sep = '\t')

In [ ]:
reviews_df.shape

In [ ]:
reviews_df

In [ ]:
reviews_df.info()

In [ ]:
reviews_df.describe()

In [ ]:
reviews_df['verified_reviews']

## Exploração dos dados

In [ ]:
sns.heatmap(reviews_df.isnull(), cbar=False);

In [ ]:
reviews_df.isnull().sum()

In [ ]:
reviews_df.hist(bins = 30, figsize=(13,5), color = 'r')

In [ ]:
reviews_df['length'] = reviews_df['verified_reviews'].apply(len)
reviews_df.head()

In [ ]:
reviews_df['length'].plot(bins = 100, kind = 'hist');

In [ ]:
reviews_df.length.describe()

In [ ]:
reviews_df[reviews_df['length'] == 2851]['verified_reviews'].iloc[0]

In [ ]:
reviews_df[reviews_df['length'] == 1]['verified_reviews'].iloc[0]

In [ ]:
reviews_df[reviews_df['length'] == 131]['verified_reviews'].iloc[0]

In [ ]:
positive = reviews_df[reviews_df['feedback'] == 1]

In [ ]:
positive

In [ ]:
positive.describe()

In [ ]:
negative = reviews_df[reviews_df['feedback'] == 0]

In [ ]:
negative

In [ ]:
negative.describe()

In [ ]:
sns.countplot(reviews_df['feedback']);

In [ ]:
sns.countplot(x = 'rating', data = reviews_df);

In [ ]:
plt.figure(figsize = (40,15))
sns.barplot(x = 'variation', y = 'rating', data = reviews_df, palette = 'deep');

In [ ]:
sentences = reviews_df['verified_reviews'].tolist()
len(sentences)

In [ ]:
print(sentences[0:10])

In [ ]:
sentences_as_one_string = ' '.join(sentences)

In [ ]:
sentences_as_one_string

In [ ]:
len(sentences_as_one_string)

In [ ]:
from wordcloud import WordCloud
plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(sentences_as_one_string));

In [ ]:
negative_list = negative['verified_reviews'].tolist()
negative_sentences_as_one_string = ' '.join(negative_list)

In [ ]:
len(negative_sentences_as_one_string)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(negative_sentences_as_one_string));

## Limpeza dos dados

In [ ]:
reviews_df.head()

In [ ]:
reviews_df = reviews_df.drop(['date', 'rating', 'length'], axis = 1)

In [ ]:
reviews_df

In [ ]:
reviews_df['variation'].unique()

In [ ]:
X_cat = reviews_df[['variation']]
X_cat

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
X_cat = onehotencoder.fit_transform(X_cat).toarray()

In [ ]:
X_cat[1], X_cat.shape

In [ ]:
type(X_cat)

In [ ]:
X_cat = pd.DataFrame(X_cat)
type(X_cat)

In [ ]:
reviews_df.drop(['variation'], axis = 1, inplace = True)

In [ ]:
reviews_df

In [ ]:
reviews_df = pd.concat([reviews_df, X_cat], axis = 1)
reviews_df

## Remoção de pontuação dos textos

In [ ]:
import string
string.punctuation

In [ ]:
test = 'Fun item to play with and get used to using. Sometimes has hard time answering the questions you ask, but I think it will be better!'

In [ ]:
test_punc_removed = [char for char in test if char not in string.punctuation]

In [ ]:
print(test_punc_removed)

In [ ]:
test_punc_removed = ''.join(test_punc_removed)
test_punc_removed

## Remoção de stop words

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

In [ ]:
len(stopwords.words('english'))

In [ ]:
print(stopwords.words('portuguese'))

In [ ]:
len(stopwords.words('portuguese'))

In [ ]:
test_punc_removed

In [ ]:
print(test_punc_removed.split())

In [ ]:
'AB'.lower()

In [ ]:
test_punc_stop_removed = [word for word in test_punc_removed.split() if word.lower() not in stopwords.words('english')]

In [ ]:
print(test_punc_stop_removed)

## Count vectorization (tokenização)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
sample_data = ['This is the first document.','This document is the second document.',
               'And this is the third one.','Is this the first document?']

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sample_data)

In [ ]:
print(vectorizer.get_feature_names_out())

In [ ]:
X

In [ ]:
print(X.toarray())

## Pipeline de limpeza dos textos

In [ ]:
def message_cleaning(message):
  text = [char for char in message if char not in string.punctuation]
  text = ''.join(text)
  text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
  return text

In [ ]:
reviews_df_clean = reviews_df['verified_reviews'].apply(message_cleaning)

In [ ]:
print(reviews_df_clean[3])

In [ ]:
print(reviews_df['verified_reviews'][3])

In [ ]:
reviews_df_clean

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer=message_cleaning)
reviews_countvectorizer = vectorizer.fit_transform(reviews_df['verified_reviews'])

In [ ]:
print(vectorizer.get_feature_names_out())

In [ ]:
len(vectorizer.get_feature_names_out())

In [ ]:
print(reviews_countvectorizer.toarray())

In [ ]:
reviews_countvectorizer.shape

In [ ]:
reviews_df

In [ ]:
reviews_df.drop(['verified_reviews'], axis = 1, inplace=True)
reviews_df

In [ ]:
type(reviews_countvectorizer)

In [ ]:
reviews = pd.DataFrame(reviews_countvectorizer.toarray())
type(reviews)

In [ ]:
reviews_df = pd.concat([reviews_df, reviews], axis = 1)

In [ ]:
reviews_df

In [ ]:
X = reviews_df.drop(['feedback'], axis = 1)
X

In [ ]:
y = reviews_df['feedback']
y

## Treinamento e avaliação do Naïve Bayes

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
y_pred = naive_bayes.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
sns.heatmap(cm, annot=True, fmt='.3g');

In [ ]:
print(classification_report(y_test, y_pred))

## Treinamento e avaliação da regressão logística

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic = LogisticRegression()
logistic.fit(X_train, y_train)

In [ ]:
y_pred = logistic.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

## Classificação de somente uma frase

In [ ]:
import pickle

In [ ]:
with open('text_classifier.pkl', 'wb') as f:
  pickle.dump([naive_bayes, onehotencoder, vectorizer], f)

In [ ]:
with open('text_classifier.pkl', 'rb') as f:
  bayes, onehot, vec = pickle.load(f)

In [ ]:
bayes

In [ ]:
onehot

In [ ]:
vec

In [ ]:
negative.head()

In [ ]:
negative_text = negative.iloc[1:2, [2,3]]
negative_text

In [ ]:
X_cat = negative_text[['variation']]
X_cat

In [ ]:
X_cat = onehot.transform(X_cat).toarray()
X_cat = pd.DataFrame(X_cat)
X_cat

In [ ]:
negative_text

In [ ]:
X_cat.index = negative_text.index

In [ ]:
X_cat.index, negative_text.index

In [ ]:
negative_df = pd.concat([negative_text, X_cat], axis = 1)
negative_df

In [ ]:
negative_df.drop(['variation'], axis = 1, inplace = True)
negative_df

In [ ]:
negative_coutvectorizer = vec.transform(negative_df['verified_reviews'])

In [ ]:
negative_coutvectorizer

In [ ]:
review = pd.DataFrame(negative_coutvectorizer.toarray())
review

In [ ]:
negative_df.drop(['verified_reviews'], axis = 1, inplace = True)
negative_df

In [ ]:
negative_df.index = review.index

In [ ]:
negative_final = pd.concat([negative_df, review], axis = 1)
negative_final

In [ ]:
bayes.predict(negative_final)
# logistic.predict(negative_final)

In [ ]:
bayes.predict_proba(negative_final)
# logistic.predict_proba(negative_final)